In [55]:
import numpy as np
import pandas as pd 
pd.set_option("max_columns",None)
from sklearn.linear_model import LinearRegression, Ridge

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import Lasso, Ridge, SGDRegressor,LinearRegression,RidgeCV,ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,RobustScaler
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor,DecisionTreeRegressor
import xgboost as xgb


pd.set_option('max_columns', 200)



In [2]:
#df = pd.ExcelFile('C:/Users/halil/Desktop/soalr_data.xlsx').parse('sheet 1')
weather_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="weather")
guneko_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="1000255-GUNEKO")

In [3]:
#data initilization
guneko_production = guneko_raw[["Date","Production"]]
guneko_gti = guneko_raw[["Date.1","GTI"]]

weather_guneko = weather_raw.loc[weather_raw.name ==1000255]
ali = pd.merge(guneko_production,weather_guneko,left_on="Date",right_on="date")
dataset = ali.drop(["name","date","lat","lon"],axis=1)

In [4]:
#encoding for not numerical datas 
dataset= dataset.set_index(["Date"])
year = pd.DataFrame(data=pd.get_dummies(dataset.index.year,prefix="year"))
month = pd.DataFrame(data=pd.get_dummies(dataset.index.month, prefix="month"))
#day = pd.DataFrame(data=pd.get_dummies(dataset.index.day,prefix="day"))
hour = pd.DataFrame(data=pd.get_dummies(dataset.index.hour,prefix="hour"))
#frames = [year,month,day,hour]
frames = [year,month,hour]
onehot_encoded = ali.join(frames)



In [5]:
#dropping useless features
onehot_encoded = onehot_encoded.drop(["Date","name","date","lat","lon"],axis=1)

In [28]:
#normalization in range (-1,1)
def normalize(df,column_list):
    for col in column_list:
        feature_range = (-1,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

In [29]:
normalized = normalize(onehot_encoded,onehot_encoded.columns[0:11])
normalized

,Production,temperature,cloud cover,global_rad:W,diffuse_rad:W,direct_rad:W,relative_humidity_2m:p,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C,sun_elevation:d,year_2020,year_2021,year_2022,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,-1.000000,-0.303571,-0.346,-1.000000,-1.000000,-1.000000,0.884332,-0.897959,-1.0,-0.312407,-0.989333,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-1.000000,-0.339286,-0.518,-1.000000,-1.000000,-1.000000,0.949527,-0.816327,-1.0,-0.342302,-0.986667,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.000000,-0.367857,-0.650,-1.000000,-1.000000,-1.000000,0.970557,-0.795918,-1.0,-0.366218,-0.873333,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-1.000000,-0.382143,-0.760,-1.000000,-1.000000,-1.000000,0.970557,-0.816327,-1.0,-0.378176,-0.733333,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-1.000000,-0.435714,0.634,-1.000000,-1.000000,-1.000000,1.000000,-0.795918,-1.0,-0.423019,-0.576000,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19031,-0.179179,-0.203571,0.644,0.146112,0.353916,-0.353991,-0.255521,-0.244898,-1.0,-0.121076,0.628000,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
19032,-1.000000,-0.160714,0.724,0.290626,0.440636,-0.229108,-0.312303,-0.285714,-1.0,-0.001495,0.698667,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
19033,-1.000000,-0.139286,0.930,0.051969,0.512826,-0.546714,-0.303891,-0.346939,-1.0,-0.028401,0.705333,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
19034,-1.000000,-0.150000,0.984,-0.277833,0.330760,-0.839906,-0.284963,-0.510204,-1.0,-0.082212,0.646667,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [30]:
#choosing specific year for train data, so other years can be drop
target = normalized.loc[normalized.year_2021 == 1]
target = target.drop(["year_2020","year_2022","year_2021"],axis=1)
target = target.reset_index(drop=True)

In [ ]:
"""train_one_year = normalized[8712:17133].reset_index(drop=True)
test_one_week = normalized[17133:17863].reset_index(drop=True)"""

In [31]:
#taking january 2022 as a dataset
aw = normalized.loc[normalized.year_2022 == 1]
aw1 = aw.loc[aw.month_1 == 1]
aw1 = aw1.reset_index(drop=True)

In [32]:
#25 day of train and 5 days of test data
train = aw1[:len(aw1)- 24*5]
test = aw1[len(aw1)- 24*5:]


x_train = train.copy()
x_test = test.copy()

y_train = x_train.pop('Production')
y_test = x_test.pop('Production')


In [21]:
from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error,explained_variance_score

def metrics(y_train1,y_train_pred1,y_test1,y_test_pred1):
    print("for train data, mean squared error is",mean_squared_error(y_train1, y_train_pred1))
    print("for train data, mean absolute error is:",mean_absolute_error(y_train1, y_train_pred1))
    print("for train data, mean absolute percentage error is",mean_absolute_percentage_error(y_train1, y_train_pred1),"\n")

    print("for test data, mean squared error is",mean_squared_error(y_test1, y_test_pred1))
    print("for test data, mean absolute error is:",mean_absolute_error(y_test1, y_test_pred1))
    print("for test data, mean absolute percentage error is",mean_absolute_percentage_error(y_test1, y_test_pred1))


In [34]:
#kneigbor model, give best test result for data 
KN = KNeighborsRegressor()
KN.fit(x_train, y_train)

KN_train_pred = KN.predict(x_train)
KN_test_pred = KN.predict(x_test)

metrics(y_train,KN_train_pred,y_test,KN_test_pred)

for train data, mean squared error is 0.025162513635248664
for train data, mean absolute error is: 0.061945880306536034
for train data, mean absolute percentage error is 0.15768836865056174 

for test data, mean squared error is 0.019375049390398073
for test data, mean absolute error is: 0.056613279946613276
for test data, mean absolute percentage error is 0.16994489387773523


In [44]:
#l2 regularization
ridge_reg = Ridge(alpha=1) 

ridge_reg.fit(x_train,y_train)

ridge_train = ridge_reg.predict(x_train)
ridge_test = ridge_reg.predict(x_test)

metrics(y_train,ridge_train,y_test,ridge_test)

for train data, mean squared error is 0.034527711462855826
for train data, mean absolute error is: 0.1220116541461929
for train data, mean absolute percentage error is 0.24271323214329446 

for test data, mean squared error is 0.023581306906930905
for test data, mean absolute error is: 0.11143830197650632
for test data, mean absolute percentage error is 0.25038221306399133


In [54]:
#l1 regularization 

lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(x_train,y_train)
lasso_train = lasso_reg.predict(x_train)
lasso_test = lasso_reg.predict(x_test)

metrics(y_train,lasso_train,y_test,lasso_test)



for train data, mean squared error is 0.136293563825218
for train data, mean absolute error is: 0.2252825036807836
for train data, mean absolute percentage error is 0.7400431882649566 

for test data, mean squared error is 0.07299252770563189
for test data, mean absolute error is: 0.19386178528255021
for test data, mean absolute percentage error is 0.8697519291343634


In [68]:
#elastic_net is both mixture of lasso and ridge regresion, it can be adjust by using l1_ratio

elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter= 5000)

elastic_net.fit(x_train,y_train)

elastic_train = elastic_net.predict(x_train)
elastic_test = elastic_net.predict(x_test)

metrics(y_train,elastic_train,y_test,elastic_test)

for train data, mean squared error is 0.0952320596391214
for train data, mean absolute error is: 0.17886564854427234
for train data, mean absolute percentage error is 0.5896638571939854 

for test data, mean squared error is 0.040163650718069537
for test data, mean absolute error is: 0.1313798201040205
for test data, mean absolute percentage error is 0.6587472768170854


In [45]:
#l2 and l1 regularizations also can be use with SGD regressor model by using penalty parameter set to "l2" or "l1"
sgd_reg = SGDRegressor(penalty="l2")

sgd_reg.fit(x_train,y_train)

sgd_train = sgd_reg.predict(x_train)
sgd_test = sgd_reg.predict(x_test)

metrics(y_train,sgd_train,y_test,sgd_test)


for train data, mean squared error is 0.045173429333708916
for train data, mean absolute error is: 0.13409039047126892
for train data, mean absolute percentage error is 0.3289186970434525 

for test data, mean squared error is 0.018629921133302422
for test data, mean absolute error is: 0.10019600410799608
for test data, mean absolute percentage error is 0.33921217249948565
